In [1]:
import pickle
import pandas as pd

In [2]:
data = pd.read_csv('iot_m_49_1.csv', engine="pyarrow")

In [3]:
data.head(10)

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label,detailed-label
0,2019-02-28 19:50:31+00:00,Cnkx4U1xJgOIPooe2,192.168.1.193,51358,134.209.72.171,80,tcp,http,1.874424,149,...,-,0,ShADadttfF,234,12994,285,418624,-,Malicious,FileDownload
1,2019-02-28 19:50:33+00:00,C5t6Kp1UnQyH7JCC45,192.168.1.193,51360,134.209.72.171,80,tcp,http,1.871487,151,...,-,0,ShADadttfF,282,15702,282,412612,-,Malicious,FileDownload
2,2019-02-28 19:50:35+00:00,CRdXqi4p3HG1obs6xl,192.168.1.193,51362,134.209.72.171,80,tcp,http,2.376374,148,...,-,0,ShADadttfF,174,9600,201,291562,-,Malicious,FileDownload
3,2019-02-28 19:50:37+00:00,Co7Sfi3KfjPoJwW5Kf,192.168.1.193,51364,134.209.72.171,80,tcp,-,3.092539,0,...,-,0,S,6,360,0,0,-,Benign,-
4,2019-02-28 19:50:44+00:00,CKaKqD34bwvqLUcID9,192.168.1.193,51364,134.209.72.171,80,tcp,http,1.359780,148,...,-,0,ShADadttfF,178,9840,196,284806,-,Malicious,FileDownload
5,2019-02-28 19:50:46+00:00,CFk7Y1SrQmKfgTlp2,192.168.1.193,51366,134.209.72.171,80,tcp,http,1.952813,151,...,-,0,ShADadtftF,220,11878,296,434528,-,Malicious,FileDownload
6,2019-02-28 19:50:48+00:00,CyLuux1ZhzNIPptfna,192.168.1.193,24159,108.116.222.190,8081,tcp,-,0.000006,0,...,-,0,S,2,80,0,0,-,Malicious,PartOfAHorizontalPortScan
7,2019-02-28 19:50:48+00:00,CA2IXnRpixOsx5fIh,192.168.1.193,24159,197.142.48.202,8081,tcp,-,0.000006,0,...,-,0,S,2,80,0,0,-,Malicious,PartOfAHorizontalPortScan
8,2019-02-28 19:50:48+00:00,C9MktJrfmw7GTH4yg,192.168.1.193,24159,197.37.170.62,8081,tcp,-,0.000002,0,...,-,0,S,2,80,0,0,-,Malicious,PartOfAHorizontalPortScan
9,2019-02-28 19:50:48+00:00,COqW6nwmLrjaVpPvc,192.168.1.193,24159,217.222.4.197,8081,tcp,-,0.000006,0,...,-,0,S,2,80,0,0,-,Malicious,PartOfAHorizontalPortScan


In [4]:
# dropping columns

df = data.drop(['ts', 'uid', 'id.orig_h', 'id.resp_h', 'service', 'local_resp', 'local_orig', 'missed_bytes', 'detailed-label', 'tunnel_parents'], axis=1)

In [5]:
df.head()

,id.orig_p,id.resp_p,proto,duration,orig_bytes,resp_bytes,conn_state,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,51358,80,tcp,1.874424,149,198998,SF,ShADadttfF,234,12994,285,418624,Malicious
1,51360,80,tcp,1.871487,151,198966,SF,ShADadttfF,282,15702,282,412612,Malicious
2,51362,80,tcp,2.376374,148,140547,SF,ShADadttfF,174,9600,201,291562,Malicious
3,51364,80,tcp,3.092539,0,0,S0,S,6,360,0,0,Benign
4,51364,80,tcp,1.359780,148,137299,SF,ShADadttfF,178,9840,196,284806,Malicious


In [6]:
# encoding categorical variables

unique_values = df['proto'].unique()
label_mapping = {value: label for label, value in enumerate(unique_values)}
df['proto'] = df['proto'].map(label_mapping)

In [7]:
unique_values = df['conn_state'].unique()
label_mapping = {value: label for label, value in enumerate(unique_values)}
df['conn_state'] = df['conn_state'].map(label_mapping)

In [8]:
df['label'] = df['label'].map({'Benign': 0, 'Malicious': 1})
df['label'] = df['label'].astype(int)

In [9]:
unique_values = df['history'].unique()
unique_values

array(['ShADadttfF', 'S', 'ShADadtftF', 'ShADadtfF', 'Sr', 'ShADadfR',
       'ShADdfrFr', '-', 'ShADdar', 'ShR', 'Dd', 'Hr', 'R', 'DdA', 'D',
       'ShADadr', 'ShADdrfFr', 'ShADdfFr', 'ShAdfR', 'ShADfdtrFr',
       'DdAFaf', 'ShADFadfR', 'ShAFf', 'ShADFafRdt', 'ShADard',
       'ShADdfFrr', 'ShADFafdtR', 'ShwR', 'ShADFaTfdtR', 'ShAfdtR',
       'ShADfdtFr', 'ShADFadfRR', 'ShADad'], dtype=object)

In [11]:
df['history'] = df['history'].replace('-', 'Hr', inplace=True)
unique_values = df['history'].unique()
label_mapping = {value: label for label, value in enumerate(unique_values)}
df['history'] = df['history'].map(label_mapping)

In [12]:
df.head(10)

,id.orig_p,id.resp_p,proto,duration,orig_bytes,resp_bytes,conn_state,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,51358,80,0,1.874424,149,198998,0,0,234,12994,285,418624,1
1,51360,80,0,1.871487,151,198966,0,0,282,15702,282,412612,1
2,51362,80,0,2.376374,148,140547,0,0,174,9600,201,291562,1
3,51364,80,0,3.092539,0,0,1,0,6,360,0,0,0
4,51364,80,0,1.359780,148,137299,0,0,178,9840,196,284806,1
5,51366,80,0,1.952813,151,209560,0,0,220,11878,296,434528,1
6,24159,8081,0,0.000006,0,0,1,0,2,80,0,0,1
7,24159,8081,0,0.000006,0,0,1,0,2,80,0,0,1
8,24159,8081,0,0.000002,0,0,1,0,2,80,0,0,1
9,24159,8081,0,0.000006,0,0,1,0,2,80,0,0,1


In [13]:
df.describe()

,id.orig_p,id.resp_p,proto,conn_state,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
count,5.410561e+06,5.410561e+06,5.410561e+06,5.410561e+06,5410561.0,5.410561e+06,5.410561e+06,5.410561e+06,5.410561e+06,5.410561e+06
mean,2.734700e+04,8.077058e+03,3.674296e-04,1.001099e+00,0.0,3.002600e+00,1.201892e+02,4.027863e-03,9.855185e-01,9.993226e-01
std,3.259144e+03,1.650950e+02,2.078420e-02,5.985420e-02,0.0,1.951891e+00,8.409645e+01,9.249058e-01,5.254039e+02,2.601769e-02
min,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1.000000e+00,4.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.415900e+04,8.081000e+03,0.000000e+00,1.000000e+00,0.0,2.000000e+00,8.000000e+01,0.000000e+00,0.000000e+00,1.000000e+00
50%,3.053500e+04,8.081000e+03,0.000000e+00,1.000000e+00,0.0,2.000000e+00,8.000000e+01,0.000000e+00,0.000000e+00,1.000000e+00
75%,3.053500e+04,8.081000e+03,0.000000e+00,1.000000e+00,0.0,4.000000e+00,1.600000e+02,0.000000e+00,0.000000e+00,1.000000e+00
max,6.070000e+04,3.053500e+04,2.000000e+00,7.000000e+00,0.0,1.796000e+03,9.375800e+04,1.792000e+03,4.345280e+05,1.000000e+00


In [14]:
min_value = df['id.orig_p'].min()
max_value = df['id.orig_p'].max()

# Perform min-max normalization on the column
df['id.orig_p'] = (df['id.orig_p'] - min_value) / (max_value - min_value)

In [15]:
min_value = df['id.resp_p'].min()
max_value = df['id.resp_p'].max()

# Perform min-max normalization on the column
df['id.resp_p'] = (df['id.resp_p'] - min_value) / (max_value - min_value)

In [16]:
# fill missing values

import numpy as np
df = df.replace('-', np.nan)

In [17]:
df = df.fillna(method="ffill")

In [18]:
for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')

In [20]:
df = df.astype(float)

In [21]:
df.dtypes

id.orig_p        float64
id.resp_p        float64
proto            float64
duration         float64
orig_bytes       float64
resp_bytes       float64
conn_state       float64
history          float64
orig_pkts        float64
orig_ip_bytes    float64
resp_pkts        float64
resp_ip_bytes    float64
label            float64
dtype: object

In [22]:
# save cleaned file

df.to_csv("cleaned_iot_m_49_1.csv", index=False)

In [23]:
# shuffle rows

df = df.sample(frac=1, random_state=42)

In [24]:
data = df

In [25]:
# load trained model

with open('model2.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [26]:
X = data.drop('label', axis=1)
y = data['label']

In [27]:
X

,id.orig_p,id.resp_p,proto,duration,orig_bytes,resp_bytes,conn_state,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes
2735030,0.397977,0.264647,0.0,0.000005,0.0,0.0,1.0,0.0,2.0,80.0,0.0,0.0
3515288,0.397977,0.264647,0.0,0.000004,0.0,0.0,1.0,0.0,2.0,80.0,0.0,0.0
4619908,0.503023,0.264647,0.0,0.000005,0.0,0.0,1.0,0.0,2.0,80.0,0.0,0.0
3203964,0.397977,0.264647,0.0,0.000008,0.0,0.0,1.0,0.0,4.0,160.0,0.0,0.0
5345560,0.503023,0.264647,0.0,0.000505,0.0,0.0,1.0,0.0,8.0,320.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1570006,0.503023,0.264647,0.0,0.000008,0.0,0.0,1.0,0.0,4.0,160.0,0.0,0.0
2234489,0.503023,0.264647,0.0,0.000005,0.0,0.0,1.0,0.0,2.0,80.0,0.0,0.0
4926484,0.503023,0.264647,0.0,0.000254,0.0,0.0,1.0,0.0,4.0,160.0,0.0,0.0
4304572,0.503023,0.264647,0.0,0.000005,0.0,0.0,1.0,0.0,2.0,80.0,0.0,0.0


In [31]:
y

2735030    1.0
3515288    1.0
4619908    1.0
3203964    1.0
5345560    1.0
          ... 
1570006    1.0
2234489    1.0
4926484    1.0
4304572    1.0
1692743    1.0
Name: label, Length: 5410561, dtype: float64

In [32]:
y_pred = loaded_model.predict(X)

In [34]:
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix, accuracy_score

accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9999595236057777


In [35]:
tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
(tn, fp, fn, tp)

(3561, 104, 115, 5406781)

In [36]:
roc_auc_score(y, y_pred, multi_class='ovr')

0.9858010980383182